In [4]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import datetime as dt
import csv

In [29]:
hoje = dt.datetime.today().strftime('%Y-%m-%d')

In [6]:
df_basetotal = pd.read_excel('input_sem_exclusao08042024_teste.xlsx', sheet_name='base_total')
df_recuperi = pd.read_excel('Base_camila_0204.xlsx', header=1)
df_acaocontrole = pd.read_csv('base_grupo_acao_controle.csv')

In [7]:
df_recuperi.rename(columns={'Número do título': 'loan_id'}, inplace=True)
df_recuperi['flag_protesto'] = 1

In [8]:
df_recuperi = df_recuperi.loc[:,['loan_id','flag_protesto']]

In [9]:
df = pd.merge(df_basetotal,df_recuperi, how='left', on='loan_id')

In [10]:
df['flag_protesto'].fillna(0, inplace=True)
df['flag_protesto'] = df['flag_protesto'].astype(int)

In [11]:
df['flag_parcela'] = ''
df['regiao'] = ''
df['custo'] = ''
df['grupo_controle'] = ''
df['acao_controle'] = ''
df['retorno'] = ''

In [12]:
df[['loan_id','valor_parcela','flag_parcela']].head(5)

,loan_id,valor_parcela,flag_parcela
0,23488783,179.85,
1,19853726,422.48,
2,23896506,535.04,
3,22407538,133.24,
4,23901789,280.35,


In [13]:
for indice, linha in df.iterrows():
    if linha['valor_parcela'] > 300:
        df.at[indice, 'flag_parcela'] = '>300'
    else:
        df.at[indice, 'flag_parcela'] = '=<300'

In [14]:
df[['loan_id','valor_parcela','flag_parcela']]

,loan_id,valor_parcela,flag_parcela
0,23488783,179.850000,=<300
1,19853726,422.480000,>300
2,23896506,535.040000,>300
3,22407538,133.240000,=<300
4,23901789,280.350000,=<300
...,...,...,...
57478,18503220,156.999953,=<300
57479,15469723,214.171655,=<300
57480,25032874,283.991250,=<300
57481,20916846,152.000000,=<300


In [15]:
df['regiao'] = df['address_state']
df['regiao'].fillna('na', inplace=True)

In [16]:
for indice, linha in df.iterrows():
    if linha['regiao'] == 'SP':
        df.at[indice, 'custo'] = 3.19
    else:
        df.at[indice, 'custo'] = 9.14

In [17]:
df_grupocontrole = df_acaocontrole.copy()

In [18]:
df_grupocontrole['flag_controle'] = '1'

In [19]:
df_grupocontrole = df_grupocontrole.loc[:,['loan_id','flag_controle','grupo']]

In [20]:
df = pd.merge(df,df_grupocontrole,how='left', on='loan_id')

In [21]:
df['flag_controle'].fillna(0, inplace=True)
df['flag_controle'] = df['flag_controle'].astype(int)

In [22]:
df['grupo_controle'] = df['flag_controle']
df['acao_controle'] = df['grupo']

In [23]:
df = df.drop(['flag_controle','grupo'], axis=1)

In [24]:
df_retorno = df_acaocontrole.copy()

In [25]:
df_retorno = df_retorno.loc[:,['loan_id','data_retorno','semana']]

In [26]:
df_retorno.head(3)

,loan_id,data_retorno,semana
0,15937649,2024-02-16,semana 01
1,6711419,2024-02-16,semana 01
2,23267037,2024-02-16,semana 01


In [31]:
df_retorno = df_retorno[df_retorno['data_retorno'] <= hoje]

In [37]:
df = pd.merge(df,df_retorno, how='left', on='loan_id')

In [39]:
df['retorno'] = df['semana']
df['retorno'].fillna(0, inplace=True)

In [40]:
df = df.drop(['data_retorno','semana'], axis=1)

In [45]:
atraso = ['2 - 30a60', '3 - 60a90', '4 - 90a120']
df_final = df[df['late_category_expandida_noverde_atual'].isin(atraso)]

In [48]:
df_final.shape

(26457, 70)

In [53]:
flag_protesto = df_final.pivot_table(index='late_category_expandida_noverde_atual', 
                             columns='flag_protesto', 
                             values='loan_id', 
                             aggfunc='count', 
                             margins=True, 
                             margins_name='Total')

In [54]:
flag_protesto

flag_protesto,0,1,Total
late_category_expandida_noverde_atual,,,
2 - 30a60,10738,13,10751
3 - 60a90,7527,931,8458
4 - 90a120,5469,1779,7248
Total,23734,2723,26457


In [55]:
df_final.head()

,dt_ref,borrower_id,loan_id,CPF,lecca_id,reference_date,safra,safra_reparc,conditions,cluster,...,fee_padrao_noverde,acordo_a_vencer,address_state,flag_protesto,flag_parcela,regiao,custo,grupo_controle,acao_controle,retorno
0,202404,4955621,23488783,3690846595,9632570.0,2024-04-01,2023-08-01,-,original,demais,...,0.02,0,SE,0,=<300,SE,9.14,0,NaN,0
1,202404,3730893,19853726,21785286862,4675629.0,2024-04-01,2023-03-01,-,original,pagou_adjusted_value,...,0.02,0,SP,1,>300,SP,3.19,1,acao,semana 05
2,202404,3730893,19853726,21785286862,4675629.0,2024-04-01,2023-03-01,-,original,pagou_adjusted_value,...,0.02,0,SP,1,>300,SP,3.19,1,acao,semana 05
3,202404,3261294,23896506,60350554358,10744377.0,2024-04-01,2023-08-01,-,original,demais,...,0.02,0,PA,1,>300,PA,9.14,1,acao,semana 05
4,202404,8643792,22407538,77023315349,7852121.0,2024-04-01,2023-06-01,2023-11,reparcelamento,demais,...,0.02,0,CE,0,=<300,CE,9.14,0,NaN,0


In [ ]:
flag_grupo = df_final.pivot_table(index='late_category_expandida_noverde_atual',
                                  columns='')